In [173]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import requests
import pprint
from bs4 import BeautifulSoup
from lxml import html
import re
import time
import sys
import datetime

In [185]:
# https://chromedriver.chromium.org/downloads
local_path_of_driver = "/Users/steffenbrockmann/Downloads/chromedriver-2"

In [184]:
def check_refund(train_number):
    driver = webdriver.Chrome(executable_path=local_path_of_driver)
    base_URL = 'https://italoinviaggio.italotreno.it/en/train'
    train_number = train_number
    driver.get(base_URL + '/' + str(train_number))
    try:
        delay = driver.find_element_by_xpath('/html/body/div[2]/section/div/div/div[1]/div/div/div[3]/span[2]').get_attribute("innerText")
        #print(delay)
        if (int(delay.split()[0])) >= 60:
            return True
        else:
            return False
    except:
        #print('There is no delay')
        return False

In [183]:
check_refund(8907)

True

In [176]:
def book_ticket(departure_station, arrival_station):
    train_URL = 'https://www.italotreno.it/en/destinations-timetable/trains-schedules'    
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(options=options, executable_path=local_path_of_driver)    
    driver.get(train_URL)
    
    search_box_from = driver.find_element_by_xpath('/html/body/main/section[3]/div[2]/form/div[1]/div/table/tbody/tr[1]/td[1]/fieldset/div/input[1]').send_keys(departure_station)
    search_box_to = driver.find_element_by_xpath('/html/body/main/section[3]/div[2]/form/div[1]/div/table/tbody/tr[1]/td[3]/fieldset/div/input[1]').send_keys(arrival_station)
    
    button = driver.find_element_by_xpath('/html/body/main/section[3]/div[2]/form/div[1]/div/table/tbody/tr[1]/td[4]/div/a')
    webdriver.ActionChains(driver).click_and_hold(button).perform()
    driver.execute_script("arguments[0].click();", button)
    time.sleep(3)
    
    time_of_departure = driver.find_element_by_xpath('/html/body/main/section[3]/div[3]/div[2]/table/tbody/tr[1]/td[1]/p[1]').get_attribute("innerText")
    time_of_arrival = driver.find_element_by_xpath('/html/body/main/section[3]/div[3]/div[2]/table/tbody/tr[1]/td[2]/p[1]').get_attribute("innerText")
    train_number = driver.find_element_by_xpath('/html/body/main/section[3]/div[3]/div[2]/table/tbody/tr[1]/td[4]/p[2]').get_attribute("innerText")
    button_book = driver.find_element_by_xpath('/html/body/main/section[3]/div[3]/div[2]/table/tbody/tr[1]/td[7]/div/div')
    driver.execute_script("arguments[0].click();", button_book)
    price = driver.find_element_by_xpath('/html/body/form/div/div[2]/div[6]/div[1]/div[2]/div/ul/div/div/li[2]/p[3]').get_attribute("innerText")
    today = datetime.date.today()
    date = (f'{today.year}-{today.month}-{today.day+1}')
    results_dict = {'date': date, 'departure_station': departure_station, 'arrival_station': arrival_station, 'time_of_departure': time_of_departure, 'time_of_arrival': time_of_arrival, 'train_number': train_number, 'price': price}
    return results_dict

In [180]:
book_ticket('Roma Termini', 'Napoli Centrale')

{'date': '2021-12-10',
 'departure_station': 'Roma Termini',
 'arrival_station': 'Napoli Centrale',
 'time_of_departure': '07:40',
 'time_of_arrival': '08:54',
 'train_number': '9903',
 'price': '45.90 €'}